***Challenge 1***

**Introducción**

La optimización de los procesos de visualización y análisis de los datos de sueño es esencial debido a la complejidad inherente de las señales fisiológicas registradas durante el sueño. Estas señales, como el EEG, EOG y EMG, pueden contener una gran cantidad de información que requiere una representación clara y comprensible para su interpretación adecuada. La visualización efectiva de los datos de sueño permitiría a los expertos en sueño identificar fácilmente los eventos relevantes, como las fases del Patrón Alternante Cíclico (CAP), lo que mejoraría la eficiencia y la precisión en el análisis de los datos.

Además, una visualización clara y comprensible de los datos de sueño facilitaría la comunicación y la colaboración entre expertos en sueño y otros profesionales de la salud. Al presentar los resultados de manera visualmente atractiva y fácil de entender, se simplificaría la interpretación de los hallazgos por parte de aquellos que no tienen experiencia en la lectura de señales fisiológicas del sueño. Esto sería especialmente beneficioso en entornos clínicos donde se necesita compartir y discutir los resultados de los estudios de sueño con otros profesionales médicos.

Referencia:Goldberger, A., Amaral, L., Glass, L., Hausdorff, J., Ivanov, P. C., Mark, R., ... & Stanley, H. E. (2000). PhysioBank, PhysioToolkit, and PhysioNet: Components of a new research resource for complex physiologic signals. Circulation [Online]. 101 (23), pp. e215–e220.

**Metodologia**

Primero, importamos la base de datos desde un archivo plano txt, extrayendo los datos a partir de la fila 23 para seleccionar únicamente aquellos archivos que contienen información. Estos datos fueron organizados en un dataframe. Posteriormente, eliminamos los valores únicos dentro de las columnas para evitar que los casos aislados afecten al modelo. Además, se contó el número de eventos y estados de sueño para cada uno, asegurando que no existieran valores nulos.

Luego, llevamos a cabo la conversión de los datos utilizando el método get_dummies. Este proceso asignó valores booleanos a todos los valores de las columnas Position y Event. Asimismo, los estados de sueño y la ubicación se codificaron en valores enteros.

Después, implementamos un modelo de regresión logística utilizando 16 características. Principalmente, se consideraron el tiempo, la duración, la ubicación, la posición y los posibles eventos. El objetivo fue predecir el "Sleep Stage".

Finalmente, dividimos los datos en conjuntos de entrenamiento y prueba, con un tamaño de datos de prueba del 20% y una semilla aleatoria de 42.

Tras evaluar el modelo, observamos un nivel de precisión del 94%.

**Resultados**

-Precision: 94.09%

-Sensibilidad: 100%

-Especifidad: 100%

-F1 score: 93.35%


**Discusiones**


La precisión del modelo alcanza un 94.09%, lo que significa que las predicciones positivas son correctas en promedio. Tanto la sensibilidad como la especificidad, ambas al 100%, indican que el modelo es capaz de distinguir  entre los falsos negativos y los falsos positivos. En otras palabras, no se pasan por alto ningún caso positivo ni se clasifican erróneamente casos negativos.

El F1-score, que integra la precisión y la sensibilidad tiene un valor del 93.35% indicando una alta fiabilidad en la capacidad del modelo para clasificar correctamente ambas clases.

**Conclusiones**

En conclusión, los resultados respaldan la eficacia del modelo en la tarea de clasificación. La  alta precisión, sensibilidad y especificidad sugiere una capacidad robusta para identificar tanto casos positivos como negativos con gran precisión y confiabilidad. Asimismo, la ausencia de anomalías en las métricas sugiere que el modelo no sufre de subajuste (underfitting) ni sobreajuste (overfitting). 

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
limit=23
data=[]

with open("n4.txt", "r") as f:
    current_row=0
    for line in f:
        current_row+=1
        if current_row>=limit:
            columns=line.strip().split("\t")
            data.append(columns)

#Creamos un DatFrame con los valors
values=pd.DataFrame(data)
#Definimos los nombres de las columnas
values.columns=["Sleep Stage","Position","Time [hh:mm:ss]","Event","Duration[s]","Location"]
#Observamos que tipo de valor es cada una.
values.dtypes


Sleep Stage        object
Position           object
Time [hh:mm:ss]    object
Event              object
Duration[s]        object
Location           object
dtype: object

In [9]:
#Casteamos los valores del datetime
values['Time [hh:mm:ss]'] = values['Time [hh:mm:ss]'].astype(str)
values["Time [hh:mm:ss]"] = pd.to_datetime('01/01/2004 '+values["Time [hh:mm:ss]"], format='%d/%m/%Y %H:%M:%S')
values.loc[values['Time [hh:mm:ss]'].dt.hour < 8, 'Time [hh:mm:ss]'] += pd.DateOffset(days=1)

values["Duration[s]"] = values["Duration[s]"].astype(int)
#Observamos los valores
values

,Sleep Stage,Position,Time [hh:mm:ss],Event,Duration[s],Location
0,W,Prone,2004-01-01 22:36:37,SLEEP-S0,30,EOG-Left-A2
1,W,Prone,2004-01-01 22:37:07,SLEEP-S0,30,EOG-Left-A2
2,W,Prone,2004-01-01 22:37:37,SLEEP-S0,30,EOG-Left-A2
3,W,Prone,2004-01-01 22:38:07,SLEEP-S0,30,EOG-Left-A2
4,W,Prone,2004-01-01 22:38:37,SLEEP-S0,30,EOG-Left-A2
...,...,...,...,...,...,...
1261,W,Prone,2004-01-02 06:59:37,SLEEP-S0,30,EOG-Left-A2
1262,W,Unknown Position,2004-01-02 07:00:07,SLEEP-S0,30,EOG-Left-A2
1263,W,Unknown Position,2004-01-02 07:00:37,SLEEP-S0,30,EOG-Left-A2
1264,W,Supine,2004-01-02 07:01:07,SLEEP-S0,30,EOG-Left-A2


In [10]:
# Elimina la columna si todos sus datos son iguales
for col in values.columns:
    if values[col].nunique() == 1:
        values.drop(col, axis=1, inplace=True)

values

,Sleep Stage,Position,Time [hh:mm:ss],Event,Duration[s],Location
0,W,Prone,2004-01-01 22:36:37,SLEEP-S0,30,EOG-Left-A2
1,W,Prone,2004-01-01 22:37:07,SLEEP-S0,30,EOG-Left-A2
2,W,Prone,2004-01-01 22:37:37,SLEEP-S0,30,EOG-Left-A2
3,W,Prone,2004-01-01 22:38:07,SLEEP-S0,30,EOG-Left-A2
4,W,Prone,2004-01-01 22:38:37,SLEEP-S0,30,EOG-Left-A2
...,...,...,...,...,...,...
1261,W,Prone,2004-01-02 06:59:37,SLEEP-S0,30,EOG-Left-A2
1262,W,Unknown Position,2004-01-02 07:00:07,SLEEP-S0,30,EOG-Left-A2
1263,W,Unknown Position,2004-01-02 07:00:37,SLEEP-S0,30,EOG-Left-A2
1264,W,Supine,2004-01-02 07:01:07,SLEEP-S0,30,EOG-Left-A2


In [11]:
values["Sleep Stage"].value_counts() #Existen 7 posibles estados de sueño

Sleep Stage
S2    565
W     223
R     199
S4    119
S3    117
S1     22
MT     21
Name: count, dtype: int64

In [12]:
values["Event"].value_counts() #Existen 9 posibles eventos

Event
SLEEP-S2     401
SLEEP-S0     223
SLEEP-REM    198
MCAP-A1      192
SLEEP-S4      76
SLEEP-S3      63
MCAP-A3       52
MCAP-A2       43
SLEEP-S1      18
Name: count, dtype: int64

In [13]:
values["Position"].value_counts()

Position
Supine              586
Right               494
Prone               126
Left                 47
Unknown Position     13
Name: count, dtype: int64

In [14]:
values['Position'] = values['Position'].replace('Unknown Position', method='ffill') # Se reemplazan los 'Unknown Position'
values["Position"].value_counts()

Position
Supine    590
Right     494
Prone     135
Left       47
Name: count, dtype: int64

In [15]:
values.dtypes

Sleep Stage                object
Position                   object
Time [hh:mm:ss]    datetime64[ns]
Event                      object
Duration[s]                 int32
Location                   object
dtype: object

In [16]:
from sklearn.preprocessing import MinMaxScaler

values['Time [hh:mm:ss]']=pd.to_numeric(values['Time [hh:mm:ss]'])

scaler = MinMaxScaler(feature_range=(0, 1))
values['Time [hh:mm:ss]'] = scaler.fit_transform(values[['Time [hh:mm:ss]']])

In [17]:
from sklearn.preprocessing import LabelEncoder
# Conversion de datos
values = pd.get_dummies(values, columns=['Position','Event'])

values["Sleep Stage"]=LabelEncoder().fit_transform(values["Sleep Stage"])
values["Location"]=LabelEncoder().fit_transform(values["Location"])

print(values.head())

   Sleep Stage  Time [hh:mm:ss]  Duration[s]  Location  Position_Left   
0            6          0.00000           30         1          False  \
1            6          0.00099           30         1          False   
2            6          0.00198           30         1          False   
3            6          0.00297           30         1          False   
4            6          0.00396           30         1          False   

   Position_Prone  Position_Right  Position_Supine  Event_MCAP-A1   
0            True           False            False          False  \
1            True           False            False          False   
2            True           False            False          False   
3            True           False            False          False   
4            True           False            False          False   

   Event_MCAP-A2  Event_MCAP-A3  Event_SLEEP-REM  Event_SLEEP-S0   
0          False          False            False            True  \
1         

In [18]:
#Observamos que tipo de valor es cada una.
values.dtypes

Sleep Stage          int32
Time [hh:mm:ss]    float64
Duration[s]          int32
Location             int32
Position_Left         bool
Position_Prone        bool
Position_Right        bool
Position_Supine       bool
Event_MCAP-A1         bool
Event_MCAP-A2         bool
Event_MCAP-A3         bool
Event_SLEEP-REM       bool
Event_SLEEP-S0        bool
Event_SLEEP-S1        bool
Event_SLEEP-S2        bool
Event_SLEEP-S3        bool
Event_SLEEP-S4        bool
dtype: object

In [19]:
valores_nulos=values.isnull().sum().sum()
valores_nulos #No existen valores nulos.

0

In [20]:
#Observamos los datos
values

,Sleep Stage,Time [hh:mm:ss],Duration[s],Location,Position_Left,Position_Prone,Position_Right,Position_Supine,Event_MCAP-A1,Event_MCAP-A2,Event_MCAP-A3,Event_SLEEP-REM,Event_SLEEP-S0,Event_SLEEP-S1,Event_SLEEP-S2,Event_SLEEP-S3,Event_SLEEP-S4
0,6,0.00000,30,1,False,True,False,False,False,False,False,False,True,False,False,False,False
1,6,0.00099,30,1,False,True,False,False,False,False,False,False,True,False,False,False,False
2,6,0.00198,30,1,False,True,False,False,False,False,False,False,True,False,False,False,False
3,6,0.00297,30,1,False,True,False,False,False,False,False,False,True,False,False,False,False
4,6,0.00396,30,1,False,True,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,6,0.99604,30,1,False,True,False,False,False,False,False,False,True,False,False,False,False
1262,6,0.99703,30,1,False,True,False,False,False,False,False,False,True,False,False,False,False
1263,6,0.99802,30,1,False,True,False,False,False,False,False,False,True,False,False,False,False
1264,6,0.99901,30,1,False,False,False,True,False,False,False,False,True,False,False,False,False


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# features y target 
features = values[['Time [hh:mm:ss]','Duration[s]','Location','Position_Left','Position_Prone','Position_Right','Position_Supine','Event_MCAP-A1','Event_MCAP-A2','Event_MCAP-A3','Event_SLEEP-REM','Event_SLEEP-S0','Event_SLEEP-S1','Event_SLEEP-S2','Event_SLEEP-S3','Event_SLEEP-S4']]
target = values['Sleep Stage']

X_train, X_test, y_train, y_test = train_test_split(values, target, test_size=0.2, random_state=42)

model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")



Accuracy: 0.9409448818897638


In [27]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
predictions = model.predict(X_test)
# Precisión
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

# Precisión
precision = precision_score(y_test, predictions, average='weighted')  # 'weighted' toma en cuenta el desbalance de clases
print(f"Precision: {precision}")

# F1-score
f1 = f1_score(y_test, predictions, average='weighted')  # 'weighted' toma en cuenta el desbalance de clases
print(f"F1-score: {f1}")

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, predictions)

# Sensibilidad 
sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
print(f"Sensibilidad (Recall): {sensitivity}")

# Especificidad
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
print(f"Especificidad: {specificity}")


Accuracy: 0.9409448818897638
Precision: 0.948388804340634
F1-score: 0.9335657339612002
Sensibilidad (Recall): 1.0
Especificidad: 1.0
